# 基于新闻文档的方案

初步结论：

- 使用默认高级api，检索不到正确结果比例较大
- 使用底层api
  - 通过嵌入检索，需要增加覆盖范围top-k，能大幅度减少检索不到正确结果问题
  - 通过llm检索，对llm依赖较大，目前比较好的是，gpt-4-turbo,其次是qwen-turbo
 
总之，使用DocumentSummaryIndex方式是有错误率的，需要权衡。

使用llm的方式太消耗token了，尤其是gpt4-turbo方式，不适用于日常使用。只适用于对比评估测试。

后续重点放在对数据的嵌入方式检索上，测试和评估大规模情况下的召回率。


## 准备

In [1]:
%%time

INDEX_PATH="news-index"
DATA_PATH="news-data"

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [2]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 6.51 ms, sys: 578 µs, total: 7.09 ms
Wall time: 109 ms


In [3]:
%%time

news_urls=[
    "https://www.guancha.cn/internation/2024_06_26_739376_s.shtml",
    "https://user.guancha.cn/main/content?id=1257049",
    "https://user.guancha.cn/main/content?id=1257033",
    "https://www.guancha.cn/politics/2024_06_26_739354.shtml",
    "https://www.guancha.cn/gaotianwei/2024_06_24_739062_s.shtml",
    "https://www.guancha.cn/economy/2024_06_26_739352.shtml"
]

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.34 µs


In [4]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data

CPU times: user 104 ms, sys: 82.4 ms, total: 187 ms
Wall time: 87.5 ms


In [5]:
%%time

import json

for news_url in news_urls:
    data=get_news_data(news_url)
    data['url']=news_url
    # a.append(data)
    # data_dict = json.loads(data)
    file_path = f'./news-data/{data["title"]}.json'
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

CPU times: user 693 ms, sys: 4.73 ms, total: 698 ms
Wall time: 2.6 s


In [6]:
%%time

from llama_index.core import(
    Document
)

def data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2.54 s, sys: 258 ms, total: 2.8 s
Wall time: 2.62 s


In [7]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir="./news-data").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

CPU times: user 17.8 ms, sys: 228 µs, total: 18 ms
Wall time: 3.03 s


In [8]:
%%time

import json

docs=[]
for document in documents:
    news_data=json.loads(document.text) # documents[0].text
    docs.append(data2doc(news_data))

documents=docs

len(docs)

CPU times: user 505 µs, sys: 67 µs, total: 572 µs
Wall time: 543 µs


6

In [6]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.76 ms, sys: 0 ns, total: 1.76 ms
Wall time: 2.74 ms


In [1]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm("gpt-3.5-turbo")
# Settings.llm=get_llm("glm-4-flash")
# Settings.llm=get_llm("yi:6b")
# Settings.llm=get_llm("qwen:14b")
# Settings.llm=get_llm("ERNIE-4.0-8K") # 有1个报错，而且很慢
# Settings.llm=get_llm("qwen-turbo") # 好用还快，只有一个测试报错
# Settings.llm=get_llm("gpt-4-turbo") # 全部通过，速度也没问题
# Settings.llm=get_llm("glm-4") # 没报错，但是有2个回答是空的
# Settings.llm=get_llm("moonshot-v1-8k") # 没有测试成功的，有1个是“内容审查拒绝”，其他的都是“当前分组上游负载已饱和，请稍后再试”
Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 3.38 s, sys: 460 ms, total: 3.84 s
Wall time: 4.13 s


In [11]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/6 [00:00<?, ?it/s]

current doc id: 丁刚：中印能实现龙象共舞吗？
current doc id: 中小银行改革加速度，今年来超80家开启合并重组模式
current doc id: 印度总理莫迪计划访问俄罗斯
current doc id: 金融监管总局：将研究提高保险资金投资创业投资基金集中度的比例上限
current doc id: 韩国电池厂火灾17名中国人遇难：有多少同胞，在韩国打工？
current doc id: 高天伟：国产可重复火箭完成10公里测试，有一点与马斯克很不一样


Generating embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 515 ms, sys: 26 ms, total: 541 ms
Wall time: 1min 31s


In [12]:
%%time

doc_summary_index.storage_context.persist("news-index")

CPU times: user 21.3 ms, sys: 0 ns, total: 21.3 ms
Wall time: 20.9 ms


## 加载索引

In [2]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="news-index")
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 269 ms, sys: 20.9 ms, total: 290 ms
Wall time: 295 ms


## 创建查询

In [4]:
%%time

from llama_index.core import get_response_synthesizer

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", 
    use_async=True,
    streaming=True
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 169 µs, sys: 0 ns, total: 169 µs
Wall time: 172 µs


In [115]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
    # choice_select_prompt=None,
    choice_batch_size=20,
    # choice_top_k=10,
    # format_node_batch_fn=None,
    # parse_choice_select_answer_fn=None,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", 
    use_async=True,
    streaming=True
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 131 µs, sys: 0 ns, total: 131 µs
Wall time: 134 µs


## 查询

In [7]:
%%time

response = query_engine.query("莫迪计划访问俄罗斯有哪些重大意义？")
response.print_response_stream()

莫迪计划访问俄罗斯的几个重大意义包括：

1. **外交关系加强**：此次访问有助于加深印度与俄罗斯之间的双边关系，特别是在政治、经济和安全领域的合作。这为两国提供了一个平台来讨论共同关心的问题，并可能在国际事务中形成更紧密的合作。

2. **能源合作**：俄罗斯是全球主要的石油和天然气供应国之一，而印度是一个能源需求大国。通过访问，莫迪可能会寻求加强与俄罗斯在能源领域的合作，包括购买更多俄罗斯能源、参与俄罗斯能源项目以及可能的投资机会。

3. **经济合作**：印度和俄罗斯之间的经济关系在过去几年中有所增长，尤其是在技术、国防、航空航天和核能等领域。此次访问可以推动更多的贸易协议和投资，促进两国的经济互补性。

4. **安全与防务**：印度和俄罗斯在军事和技术领域有长期的合作历史。莫迪可能利用这次访问来讨论加强双边军事合作的可能性，包括武器装备采购、联合军演以及国防工业合作等。

5. **地缘政治影响**：此次访问也可能涉及对全球地缘政治格局的影响。印度作为多边体系中的重要力量之一，其与俄罗斯的紧密关系可以为两国在国际组织和论坛中提供更大的影响力，并可能在某些地区冲突或战略问题上采取一致立场。

6. **应对挑战**：面对当前国际环境的不确定性（如美国对俄罗斯的制裁、全球能源市场波动等），印度寻求多元化的合作伙伴关系，以确保其经济安全和地缘政治利益。与俄罗斯的合作可以提供一个稳定的战略伙伴，帮助印度在多变的世界中保持平衡。

7. **文化交流与教育**：访问也可能包括文化合作和教育交流，促进两国人民之间的相互理解和友谊。这有助于加强民间外交，并为未来的技术、学术和艺术领域的合作铺平道路。CPU times: user 669 ms, sys: 26.3 ms, total: 695 ms
Wall time: 29.7 s


In [14]:
response.source_nodes[0].metadata['url']

'https://user.guancha.cn/main/content?id=1257033'

In [117]:
%%time

response = query_engine.query("有多少同胞在韩国打工？")
response.print_response_stream()

在韩国就业的中国人有37.2万人。CPU times: user 91.5 ms, sys: 11 ms, total: 103 ms
Wall time: 12.1 s


In [121]:
%%time

response = query_engine.query("韩国电池厂火灾有哪些信息？")
response.print_response_stream()

韩国京畿道华城市的Aricell电池工厂发生了一起严重火灾，时间是2024年6月24日上午10点31分。这场火灾造成至少23人死亡，其中包括17名中国籍人员，5名韩国籍人员和1名老挝籍人员。火灾持续了大约8个小时，由160名消防员和60台消防车辆及装备共同扑灭。起火原因是工厂二楼存放的约3.5万块锂电池突然冒出大量浓烟并迅速引发大火。此外，这家工厂的管理相当混乱，没有明显的安全标识和消防设施，且工厂中的中国员工大多是非正式员工，工资按照韩国最低工资标准支付，很多人是日结工资。在没有工作时，这些员工会被解雇，且没有任何赔偿。CPU times: user 242 ms, sys: 41.2 ms, total: 283 ms
Wall time: 39.8 s


In [122]:
%%time

response = query_engine.query("国产可重复使用火箭有啥进展？")
response.print_response_stream()

国产可重复使用火箭的研制进展迅速，涉及多个企业和技术验证。航天科技集团八院最近成功进行了一次10公里级的飞行试验，使用了3台变推液氧甲烷发动机，实现了受控下降和精准的垂直软着陆，为2025年计划的4米级重复使用运载火箭首飞奠定了技术基础。同时，蓝箭航天的朱雀三号VTVL-1火箭也已成功完成300米级别的垂直起降回收验证飞行，并计划进行10公里级别的试验。这些成就表明中国在可重复使用火箭领域已经取得了显著的技术突破和实际应用的基础。CPU times: user 190 ms, sys: 39.8 ms, total: 230 ms
Wall time: 25.4 s


In [123]:
%%time

response = query_engine.query("将研究提高保险资金投资创业投资基金集中度的比例上限主要讲了啥？")
response.print_response_stream()

金融监管总局计划研究提高保险资金投资创业投资基金集中度的比例上限，以更好地引导保险资金和相关资管机构在依法合规、风险可控的前提下，加大对创业投资基金的配置力度，从而积极促进创业投资的高质量发展。CPU times: user 105 ms, sys: 7.24 ms, total: 112 ms
Wall time: 10.1 s


In [124]:
%%time

response = query_engine.query("印度总理莫迪计划在赢得第三个任期后做些啥？")
response.print_response_stream()

印度总理莫迪计划在赢得第三个任期后推出一系列亲商改革，旨在提升印度制造业的竞争力，与中国制造业展开竞争。此外，莫迪政府未来的重点将是到2047年将印度在全球制造业中的份额提高到10%。CPU times: user 94 ms, sys: 8.94 ms, total: 103 ms
Wall time: 7.72 s


In [94]:
len(response.source_nodes)

3

In [95]:
response.source_nodes[0].metadata

{'title': '印度总理莫迪计划访问俄罗斯',
 'publish_time': '2024-06-26 18:54:14',
 'author': '林兆楠',
 'url': 'https://www.guancha.cn/internation/2024_06_26_739376_s.shtml',
 'images': ['https://i.guancha.cn/news/dfic/2024/06/26/20240626173214826.jpg',
  'https://i.guancha.cn/news/external/2024/06/26/20240626172837420.png']}